# FunOS Malloc report

This notebook summarizes mcache metrics.

In [ ]:
# *NOTE*: run this command to clean output cell and meta data.

#nb-clean clean  ./funos_stats_analysis/malloc_report.ipynb   

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.ticker as mticker
from plotly.subplots import make_subplots

import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

plt.rcParams['figure.figsize'] = [15, 25]

#https://stackoverflow.com/questions/36288670/how-to-programmatically-generate-markdown-output-in-jupyter-notebooks
from IPython.display import display, Markdown, Latex

In [ ]:
DEBUG=False
RL_PLOT=False

In [ ]:
if DEBUG:
    df_dir = "../outputs/2022-07-28/11-05-49/"
    df_filename_pattern_coh = "{}malloc_caches_slot_stats_slot_{}.pkl"
    df_filename_pattern_non_coh = "{}malloc_caches_slot_stats_slot_non_coh_{}.pkl"
else:
    df_filename_pattern_coh = "malloc_caches_slot_stats_slot_{}.pkl"
    df_filename_pattern_non_coh = "malloc_caches_slot_stats_slot_non_coh_{}.pkl"


In [ ]:
slots = [s for s in range(6,16)]
slot_idx = [i for i in range(len(slots))]

In [ ]:
if DEBUG:
    df_filenames_coh = [df_filename_pattern_coh.format(df_dir, s) for s in slots]
    df_filenames_non_coh = [df_filename_pattern_non_coh.format(df_dir, s) for s in slots]
else:
    df_filenames_coh = [df_filename_pattern_coh.format(s) for s in slots]
    df_filenames_non_coh = [df_filename_pattern_non_coh.format(s) for s in slots]

In [ ]:
dfs_coh = [pd.read_pickle(df_filename) for df_filename  in df_filenames_coh]
dfs_non_coh = [pd.read_pickle(df_filename) for df_filename  in df_filenames_non_coh]

In [ ]:
def plot_vp_index(df, col, title):
    plt.title(title)
    plt.scatter(df.index, df[col])
    plt.xticks(rotation = 70)
    plt.grid()
    plt.show()

In [ ]:
def plot_vp_index_all_slots(dfs_coh, col, title, mem_type, height=30):
    note_str = '## y axis: **{}** **{}**'.format(mem_type, title)
    display(Markdown(note_str))
    note_str = "## x axis: ** vp number (`cluster.core.vp`)**"
    display(Markdown(note_str))

    # use short notation for cluster.core for x-axis
    short_vp_names = ["{}.{}".format(vp.split('_')[1], vp.split('_')[2]) for vp in dfs_coh[0].index]

    # prepare subplot
    fig, axs = plt.subplots(len(slots),1)
    fig.set_figheight(height)

    for i, s in zip(slot_idx, slots):
        df = dfs_coh[i]
        if col == "Avail":
            max_val = df['Max'][0]
            # axs[i].plot(df.index, df['Max'], 'r', label='Max')
            axs[i].axhline(y=max_val, color='r', linestyle='-') #, label="Max(limit)")
            axs[i].scatter(df.index, df[col], label=col)

            # axs[i].plot(df.index, df['Avail(avg)'], color='y', label="Avail(avg)")
            # axs[i].errorbar(df.index, df['Avail(avg)'], yerr=df['Avail(std)'], color='y', fmt=".") #, label="Avail(std)")

            # axs[i].axhline(y=df[col].mean(), color='y', linestyle='-', label="Avail(Mean)")
            # axs[i].errorbar(df.index, df[col], yerr=df[col].std(), fmt="-", label="Avail(std)")

            axs[i].plot(df.index, df['Avail(max)'], 'g', label='Avail(max)')
            # 'Avail(min) is reset to max, so if == max, then not yet updated so set to itself
            avail_min = np.where(df['Avail(min)'] != df['Max'], df['Avail(min)'], df[col])
            # avail_min = df['Avail(min)']
            axs[i].plot(df.index, avail_min, 'b', label='Avail(min)')

            # axs[i].plot(df.index, df['Repl_th_val'], '.', label='Repl_th_val')

            axs[i].set_ylim([0, max_val*1.05])
            if i == 0: print("Red line: max")
        elif col in ['Hit']:
            if i == 0:
                note_str = "*NOTE*: {}, y axis is $\log$ scale.".format(col)
                display(Markdown(note_str))
            axs[i].scatter(df.index, df[col], label="Hit (log scale)")
            axs[i].set_yscale('log')
        else:
            axs[i].scatter(df.index, df[col], label=col)
        # axs[i].scatter(df.index, df_1[col])
        axs[i].set_ylabel("Slot: {}".format(s))
        axs[i].grid()
        axs[i].legend()
        ticks_loc = axs[i].get_xticks()
        axs[i].xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
        axs[i].set_xticklabels(short_vp_names, rotation=70)
        axs[i].set_xlabel('cluster.core')
    
    plt.show()

## Malloc stats for all slots

In [ ]:
print("Column list: {}".format(dfs_coh[0].columns))

In [ ]:
# black_list = ['Avail Bytes', 'Avail(max)', 'Avail(min)', 'Max repl. time (nsec)', 'MIN repl. miss inter dur. (nsec)',
#        'MIN alloc inter dur. (nsec)', 'Max', 'val_func']
# print("Will plot except these columns: {}".format(black_list))

In [ ]:
white_list = ['Avail', 'Hit', 'Miss']
print("Will plot except these columns: {}".format(white_list))

In [ ]:
# plot coh
for col in dfs_coh[0].columns:
    # if col not in black_list: 
    if col in white_list:
        plot_vp_index_all_slots(dfs_coh, col, col, "coh")

In [ ]:
# plot non_coh
for col in dfs_non_coh[0].columns:
    # if col not in black_list: 
    if col in white_list:
        plot_vp_index_all_slots(dfs_non_coh, col, col, "non_coh")

## Value function visualization

In [ ]:
# t1 = dfs_coh[0]['val_func'][0]

In [ ]:
# type(t1)

In [ ]:
def _get_values_for_meshplot(Q):
    x = np.linspace(0, 7, 8).astype(int)
    y = np.linspace(0, 7, 8).astype(int)
    xlabel = "State1, avail"
    ylabel = "State2, avail_diff offset by 4"

    def f(x, y):
        div = 1
        return np.argmax(Q[x // div, y // div, :], axis=2)

    return x, y, xlabel, ylabel, f

In [ ]:
def _plot_Q_matplot(
    Q, title
):

    # st = config["agent"].get("state_type", "REQ_CUR")
    x, y, xlabel, ylabel, f = _get_values_for_meshplot(Q)

    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)

    fig = plt.figure()
    ax = plt.axes(projection="3d")
    # map the data to rgba values from a colormap
    ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap="viridis", edgecolor="none",antialiased=True)
    ax.set(
        xlabel=xlabel, ylabel=ylabel, zlabel="Action", title="Tabular Q values: {}".format(title)
    )

    plt.tight_layout()

    plt.show()



In [ ]:
def _plot_Q_plotly(
    Q, title
):

    x, y, xlabel, ylabel, f = _get_values_for_meshplot(Q)

    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)

    # https://plotly.com/python/3d-surface-plots/
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
    # fig = go.Figure(data=[go.Surface(z=Z)])
    fig.update_traces(
        contours_z=dict(
            show=True, usecolormap=True, highlightcolor="limegreen", project_z=True
        )
    )

    # max_x = (
    #     config["agent"]["max_request_mem"] - config["agent"]["min_request_mem"] + 1
    # )
    # offset_x = -config["agent"]["min_request_mem"]


    # max_z = config["agent"]["action_max_mem"] - config["agent"]["action_min_mem"] + 1
    # offset_z = -config["agent"]["action_min_mem"]
    # max_y = config["agent"]["max_cached_mem"] + 1
    # skip = 2

    # x_val = [i for i in range(0, max_x, skip)]
    # if st == "REQ_CUR":
    #     x_tick = [v - offset_x for v in x_val]
    # elif st == "CUR_PREV" or st == "CUR_PREV_QUANTIZED":
    #     x_tick = x_val

    y_val = [i for i in range(0, 8)]
    z_val = [i for i in range(0, 2)]
    # z_tick = [v - offset_z for v in z_val]

    zlabel = "Action"
    title = "Tabular Q values: {}".format(title)
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title=xlabel,
            yaxis_title=ylabel,
            zaxis_title=zlabel,
            # xaxis=dict(ticktext=x_tick, tickvals=x_val),
            # xaxis=dict(ticktext=x_val, tickvals=x_val),
            # yaxis=dict(ticktext=y_val, tickvals=y_val),
            # zaxis=dict(ticktext=z_tick, tickvals=z_val),
        ),
        scene_aspectmode="cube",
    )

    fig.show()

In [ ]:
def _plot_heatmap(df, title):
    max_val_arr = np.argmax(df, axis=-1)
    fig, ax = plt.subplots()
    ax.set_title(title)
    # y : state1
    # x : state2
    ax.set_ylabel("State1, avail")
    ax.set_xlabel("State2, avail_diff offset by 4")
    # sns.set(rc={'figure.figsize':(4,4)})
    sns.heatmap(max_val_arr, annot=True, ax=ax)

In [ ]:
# dfs_coh[0].index[0]

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})

In [ ]:
if RL_PLOT:
    # slot = 0
    for slot in range(10):
    # for slot in range(1):
        for i in range(len(dfs_coh[0])):
            df_i = dfs_coh[slot]['val_func'][i]
            # _plot_Q_matplot(df_i, "slot {}, vp {}".format(slot, dfs_coh[slot].index[i]))
            # _plot_Q_plotly(df_i, "slot {}, vp {}".format(slot, dfs_coh[slot].index[i]))
            _plot_heatmap(df_i, "slot {}, vp {}".format(slot, dfs_coh[slot].index[i]))

In [ ]:
# max_val_arr = np.argmax(t1, axis=-1)

In [ ]:
# max_val_arr

In [ ]:
# fig, ax = plt.subplots()
# sns.set(rc={'figure.figsize':(4,4)})
# sns.heatmap(max_val_arr, annot=True, ax=ax)

In [ ]:
# _plot_Q_plotly(t1, "test")